# Refine 3DVA
This notebook imports selected frames from a 3DVA Display job and imports them as unique volumes, then queues a Non-Uniform Refinement of each imported volume.

In [11]:
from cryosparc.tools import CryoSPARC
import json
import numpy as np
from pathlib import Path
import zipfile

with open(Path('~/release-info.json').expanduser(), 'r') as f:
    instance_info = json.load(f)

cs = CryoSPARC(**instance_info)
assert cs.test_connection()

project_number = "P60"
workspace_number = "W2"
job_number = "J7"

project = cs.find_project(project_number)
workspace = project.find_workspace(workspace_number)
var_job = project.find_job(job_number)

Connection succeeded to CryoSPARC command_core at http://cryoem0.sbi:62062
Connection succeeded to CryoSPARC command_vis at http://cryoem0.sbi:62063
Connection succeeded to CryoSPARC command_rtp at http://cryoem0.sbi:62065


First, we unzip and import the volumes as per [this example](https://tools.cryosparc.com/examples/connect_series_to_class3D.html).
However, instead of loading all volumes in one job, we load only the volumes we are interested in refining.

In [3]:
vol_series = var_job.load_output('series_0')
vol_series_path = project.dir() / vol_series['series/path'][0]
unzip_path = vol_series_path.with_suffix("")

with zipfile.ZipFile(vol_series_path, 'r') as z:
    z.extractall(unzip_path)

In [8]:
frames_of_interest = [0, 4, 5, 6]

In [10]:
lane = "cryoem9"

vol_import_jobs = {}

for f_num in frames_of_interest:

    vol_import_jobs[f_num] = workspace.create_job(
        "import_volumes",
        params = {
            "volume_blob_path": str(unzip_path / f"*{f_num:0>3}.mrc")
        },
        title = f"Frame {f_num:0>3}"
    )
    vol_import_jobs[f_num].queue(lane)
    
vol_import_jobs[f_num].wait_for_done()

'completed'

Now we queue the refinement jobs.

In [16]:
refinement_jobs = {}

for f_num in frames_of_interest:

    refinement_jobs[f_num] = workspace.create_job(
        "nonuniform_refine_new",
        connections = {
            "particles": (job_number, f"particles_series_0_frame_{f_num}"),
            "volume": (vol_import_jobs[f_num].doc["uid"], "imported_volume_1")
        },
        params = {
            # set as appropriate
            "refine_res_init": 10
        },
        title = f"Frame {f_num:0>3}"
    )
    refinement_jobs[f_num].queue(lane)

refinement_jobs[f_num].wait_for_done()

'completed'